In [19]:
!pip install gql openpyxl pandas tqdm dateparser aiohttp

  Using cached aiohttp-3.8.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached frozenlist-1.3.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (149 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)


In [1]:
ENDPOINT = "http://localhost:8088/resources/"

In [2]:
import dateparser
import csv
from functools import partial
import pandas as pd
import enum

In [3]:
def naive_date_to_date_string(date):
    return dateparser.parse(date).strftime("%Y-%m-%d")
def naive_date_to_isoformat(date):
    return dateparser.parse(date).isoformat()

class RiskStatus(enum.Enum):
    NotAtRisk = "NotAtRisk"
    AtRisk = "AtRisk"
    
def concept_lookup(field, value):
    if field == "risk_status":
        return RiskStatus.NotAtRisk.name if pd.isna(value) or not value else RiskStatus.AtRisk.name
concept_lookup("risk_status", 0)

'NotAtRisk'

In [4]:
mapping = {}
with open("../docker/mapping_file.csv", "r") as fd:
    reader = csv.DictReader(fd)
    mapping = {row["Name_hed"]: row["map_name_arches"] for row in reader}
    fd.seek(0)
    next(reader)
    types = dict(**__builtins__.__dict__)
    types.update({
        "date": naive_date_to_date_string,
        "datetime": naive_date_to_isoformat,
        "concept": concept_lookup
    })
    mapping_types = {row["map_name_arches"]: ((row.get("Type") or "str") if row["Type"] != "concept" else row["map_name_arches"], types.get(row.get("Type") or "str") if row["Type"] != "concept" else partial(types.get(row["Type"]), row["map_name_arches"])) for row in reader}
    
print(mapping)

{'DescribedBy': 'actor', 'Area': 'area_names', 'Map12500': 'area_reference_value', 'LastDescribed': 'associated_actor_display_date', 'DateOfInput': 'associated_actor_start_date', 'InputBy': 'authorizer_name', 'Barony': 'building_name', 'Parish': 'building_number', 'SourcePictFile': 'caption', 'Category': 'comment', 'DateEdited': 'compilation_start_date', 'LastEditedBy': 'compiler_name', 'County': 'county', 'AllGenPeriods': 'cultural_period', 'DateGISUpdated': 'date_of_authorization', 'SiteSummary': 'description', 'ReasonsforScheduling': 'designation_name', 'DateScheduled': 'designation_start_date', 'LegalAction': 'display_date', 'B_FileNo': 'external_cross_reference', 'DANIFarmNo': 'external_cross_reference_description', 'Altitude': 'feature_shape', 'MainPictureFile': 'images', 'DescripScheduledArea': 'land_use_notes_value', 'SMRNo - based on map sheets and site number ': 'legacy_id', 'AllTownlands': 'locality', 'WardenConditionRptSummary': 'location_description', 'SiteName': 'monument

In [5]:
def camel(string):
    string = (string[0].lower() + string.replace("_", " ").title()[1:]).replace(" ", "")
    if string[-1] != string[-1].lower():
        string += "_"
    return string

In [6]:
import pandas as pd
import tqdm

In [7]:
df = pd.read_excel("../data/dbo.tblMonument.xlsx")

/tmp/philtweir/ipykernel_35556/1192633430.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df = pd.read_excel("../data/dbo.tblMonument.xlsx")


In [8]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

# Select your transport with a defined url endpoint
transport = AIOHTTPTransport(url=ENDPOINT)

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

# Provide a GraphQL query
get_query = gql(
    """
    query Monument {
      monument {
        monumentName
      }
    }
"""
)

# Execute the query on the transport

async def get_monuments():
    return await client.execute_async(get_query)
#await get_monuments()

In [9]:
skip = [
    "cultural_period",
    "county",
    #"date_of_authorization",
    #"designation_start_date",
    "feature_shape",
    "named_location",
    "national_grid_reference",
    "resourceid",
    "primary_reference_number",
    "shine_significance",
    "spatial_accuracy_qualifier",
    "ownership",
    #"compilation_start_date",#
    "actor",
    "images",
    #"display_date",
    #"associated_actor_start_date",
    #"associated_actor_display_date",
    #"risk_status",
]

def map_row(row):
    return {mapping[key]: str(val) for key, val in row.items() if key in mapping and pd.notna(val) and mapping[key] not in skip}
async def create_monument_from_row(row):
    mapped = map_row(row)
    return await create_monument(**mapped)
#await create_monument_from_row(df.iloc[0])

In [10]:
async def bulk_create_monuments_from_rows(rows):
    mapped = [
        map_row(row)
        for ix, row in rows.iterrows()
    ]
    return await bulk_create_monument(mapped)
#await create_monument_from_row(df.iloc[0])

In [11]:
type_to_type = {
    "str": "String",
    "date": "String",
    "datetime": "String",
    "risk_status": "RiskStatus"
}
def get_type(field):
    return type_to_type[mapping_types[field][0]]

maplist = list(mapping.items())
async def create_monument(**kwargs):
    kwmapping = [v for k, v in maplist if v in kwargs and v not in skip]
    #print(list(kwmapping)[n-1])

    query = gql(
        """
        
        mutation createMonument(%s) {
          createMonument(%s) {
            ok
            monument { id }
          }
        }
    """ % (
        ", ".join(f"${camel(field)}: {get_type(field)}" for field in list(kwmapping)),
        ", ".join(f"{camel(field)}: ${camel(field)}" for field in list(kwmapping)),    
    ))

    # Execute the query on the transport
    kwargs = {
        camel(key): mapping_types[key][1](val)
        for key, val
        in kwargs.items()
        if key in kwmapping
    }
    print(kwargs)
    #print(kwargs["resourceid"])
    return await client.execute_async(query, variable_values=kwargs)

In [12]:
maplist = list(mapping.items())
async def bulk_create_monument(field_sets):
    kwmapping = [v for k, v in maplist if  v not in skip]
    #print(list(kwmapping)[n-1])

    query = gql(
        """
        mutation bulkCreateMonument($input: [MonumentInput]) {
            bulkCreateMonument(fieldSets: $input) {
                ok,
                monuments { id }
            }
        }
        """
    )
    #query = gql(
    #    """
    #    mutation {
    #        bulkCreateMonument(fieldSets: [{
    #    """ +
    #    "\n}], [{\n".join(
    #        ",\n ".join(f"{camel(field)}: \"{value}\"" for field, value in field_set.items() if field in kwmapping)
    #        for field_set in field_sets
    #    ) +
    #    """
    #        }])
    #    }
    #""")

    # Execute the query on the transport

    
    #print(kwargs["date_of_authorization"])
    #print(kwargs["resourceid"])
    values = [
            {camel(field): mapping_types[field][1](value) for field, value in field_set.items() if field in kwmapping}
            for field_set in field_sets
        ]
    return await client.execute_async(query, variable_values={
        "input": values
    })

In [13]:
import numpy as np
split_df = np.array_split(df, len(df) // 30)

In [14]:
len(df) / 8 * 50 / 60 / 60

28.984375

In [15]:
df[pd.notna(df["ProportionSiteScheduled"])]

,MonID,County,SMRNo,Council,Town,Barony,Parish,Condition,SiteName,SiteSummary,...,InternalorPublic,DANIFarmNo,WardenConditionRptSummary,DescripScheduledArea,AllTownlands,SiteManagement,DateGISUpdated,AllGenPeriods,DateGISUpdate,Category
166,167,ANT,002:003,30.0,NaN,DUNLUCE LOWER,DUNLUCE,2.0,DUNLUCE CASTLE,Dun name and souterrain suggest Early Christia...,...,Public,NaN,Refer to table below,"A line runs from the corner A, following the e...",DUNLUCE,NaN,2009-08-19 16:39:40.640,POST-MED; MEDIEVAL; LATE-MED; E.CHRIST.,NaN,4.0
168,169,ANT,002:005,30.0,NaN,DUNLUCE LOWER,DUNLUCE,3.0,CHURCH OF DUNKELISP,This site consists of post-medieval church rui...,...,Public,NaN,13/04/2007 No change since last visit in 2005....,The scheduled area is outlined in red on the m...,DUNLUCE,NaN,2010-01-15 11:00:56.137,POST-MED; MED/L.MED,NaN,4.0
171,172,ANT,002:008,30.0,NaN,DUNLUCE LOWER,DUNLUCE,2.0,DUNLUCE,Remains of an extensive settlement complex con...,...,NaN,NaN,Historical Data - Refer to table below. \n14/0...,"A line runs from the corner A, following the e...",DUNLUCE,NaN,2009-08-19 16:39:43.907,LATE-MED; POST-MED,NaN,3.0
186,187,ANT,003:001,30.0,128.0,DUNLUCE LOWER,DUNLUCE,1.0,LISSANDUFF EARTHWORKS,"This site, with ANT 003:002, consists of 2 lar...",...,Public,NaN,09/02/2007 Monuments are well maintained with...,The area to be scheduled is outlined in red on...,BUSHFOOT OR LISANDUFF,NaN,2009-08-19 16:39:40.250,PREHISTORIC,NaN,3.0
190,191,ANT,003:005,30.0,NaN,CARY,BILLY,3.0,CARNKIRK CASTLE,"On high ground, encroached on to NW by a farmy...",...,NaN,NaN,08/10/2008 [For Field Monument Warden Reports ...,Map showing outlined in red the Historic Monum...,CARNKIRK,23/8/2000\nThe warden is to discourage the nei...,2009-08-19 16:39:36.407,PREHISTORIC,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16572,16607,ANT,063:501,33.0,NaN,MASSEREENE UPPER,MAGHERAGALL,1.0,NaN,"Situated north-west of Lisburn, close to an in...",...,NaN,NaN,NaN,The line follows the base edge of the chimney ...,MONEYBROOM,NaN,NaT,C19TH,NaN,4.0
16573,16608,ANT,063:500,33.0,NaN,MASSEREENE UPPER,MAGHERAGALL,1.0,NaN,"Situated north-west of Lisburn, this intact ba...",...,NaN,NaN,NaN,The scheduled area is outlined in red on the a...,MONEYBROOM,NaN,NaT,C18TH/C19TH,NaN,4.0
16574,16609,ANT,061:502,29.0,NaN,BELFAST UPPER,SHANKILL,NaN,"Clarendon Dock (Southerly), Ritchies Dock",The southern dock is the earliest extant grav...,...,NaN,NaN,NaN,The scheduled area is outlined in red on the a...,TOWN PARKS (BELFAST),NaN,NaT,C18TH/C19TH,NaN,1.0
16575,16610,ANT,061:501,29.0,NaN,BELFAST UPPER,SHANKILL,NaN,Clarendon Dock (Northern),The northern dock was constructed by Messrs He...,...,NaN,NaN,NaN,The scheduled area is outlined in red on the a...,TOWN PARKS (BELFAST),NaN,NaT,C19TH,NaN,1.0


In [16]:
await bulk_create_monuments_from_rows(df.iloc[166:168])

TransportQueryError: {'message': "Variable '$input' got invalid value {'buildingName': 'DUNLUCE LOWER', 'buildingNumber': 'DUNLUCE', 'monumentName': 'DUNLUCE CASTLE', 'description': 'Dun name and souterrain suggest Early Christian period occupation on the headland. Despite C13th and C14th records of...uently despoiled and finally abandoned by the end of the century. See SM7 for detailed description, plans, photos etc.', 'areaReferenceValue': 'IG6', ..., 'landUseNotesValue': 'A line runs from the corner A, following the edge of the road, the edge of the access lane (taking in the area of the...ide of the stream and then the HWMMT back to C, where it turns south-eastward, following the field boundary back to A.', 'locality': 'DUNLUCE', 'dateOfAuthorization': '2009-08-19', 'comment': '4.0'} at 'input[0]'; Field 'buildingName' is not defined by type 'MonumentInput'.", 'locations': [{'line': 1, 'column': 29}]}

In [ ]:
from tqdm.notebook import tqdm
results = []
it = tqdm(total=len(df))
for rows in split_df:
    #rows = rows.iloc[37:39]
    try:
        new_results = [mon["id"] for mon in (await bulk_create_monuments_from_rows(rows))["bulkCreateMonument"]["monuments"]]
    except Exception as e:
        raise
        for k, row in rows.iterrows():
            try:
                await create_monument_from_row(row)
            except:
                print(map_row(row))
                raise
    results += new_results
    it.update(len(rows))

In [ ]:
# Provide a GraphQL query
search_query = gql(
    """
    query searchMonument($text: String, $fields: [String]) {
      searchMonument(text: $text, fields: $fields) {
          id
          monumentName
      }
    }
"""
)

# Execute the query on the transport

async def search_monuments(**kwargs):
    kwargs = {
        camel(key): val
        for key, val
        in kwargs.items()
    }
    return await client.execute_async(search_query, variable_values=kwargs)

In [17]:
await create_monument(monumentName="My Monument")

GraphQLSyntaxError: Syntax Error: Expected '$', found ')'.

GraphQL request:3:33
2 |         
3 |         mutation createMonument() {
  |                                 ^
4 |           createMonument() {

In [ ]:
await get_monuments()

In [135]:
delete_query = gql(
    """
    mutation deleteMonument($id: UUID) {
      deleteMonument(id: $id) {
        ok
      }
    }
"""
)

# Execute the query on the transport
async def delete_monument(**kwargs):
    kwargs = {
        camel(key): val
        for key, val
        in kwargs.items()
    }
    return await client.execute_async(delete_query, variable_values=kwargs)

In [7]:
for monument in (await search_monuments(text="My Monument"))["searchMonument"]:
    await delete_monument(id=monument["id"])

In [27]:
await search_monuments(text="Potter", fields=["monument_name"])

TransportQueryError: {'message': 'An unknown error occurred.', 'locations': [{'line': 2, 'column': 3}], 'path': ['searchMonument']}

---

In [11]:
df.head()

,MonID,County,SMRNo,Council,Town,Barony,Parish,Condition,SiteName,SiteSummary,...,InternalorPublic,DANIFarmNo,WardenConditionRptSummary,DescripScheduledArea,AllTownlands,SiteManagement,DateGISUpdated,AllGenPeriods,DateGISUpdate,Category
0,1,ANT,004:010,30.0,NaN,CARY,BALLINTOY,2.0,"STONE CIRCLE, TUMULUS",This site consists of a sub-circular mound 19....,...,NaN,NaN,18/02/2009 While there is a trail through the ...,The area to be scheduled lies within a circle ...,WHITE PARK,NaN,2009-08-19 16:39:37.033,PREHISTORIC,NaN,3.0
1,2,ANT,004:011,30.0,NaN,CARY,BALLINTOY,3.0,"CLOUGHNABOGHIL, DRUIDS STONE, CLOGHNABOGHIL, C...",On top of an enimence on a steep hill slope. T...,...,NaN,NaN,18/02/2009 The stones of this monument are und...,The area to be scheduled lies within a circle ...,LEMNAGH BEG,24/8/00\nMonitor burrow and whins and cut whi...,2009-08-19 16:39:33.080,PREHISTORIC; NEOLITHIC,NaN,3.0
2,3,ANT,004:012,30.0,NaN,CARY,BALLINTOY,1.0,CLEGNAGH CROMLECH,Situated on edge of ridge at edge of disused q...,...,NaN,NaN,28/04/2009 This was a re-visit to the site to ...,The area to be scheduled lies within a circle ...,CLEGNAGH,The owner will consider the FMW's suggestion t...,2010-01-15 11:00:56.107,NEOLITHIC; PREHISTORIC,NaN,3.0
3,4,ANT,004:013,30.0,NaN,CARY,BALLINTOY,2.0,"DRUID'S STONE, MOUNT DRUID, DRUID STONE, THE D...",On top of a knoll with superb views over White...,...,NaN,NaN,03/03/2009 The stones of this monument are und...,The area to be scheduled is a circle 15m in di...,BALLINTOY DEMESNE; MAGHERABOY,NaN,2010-01-15 11:09:17.070,PREHISTORIC; NEOLITHIC,NaN,3.0
4,5,ANT,004:014,30.0,NaN,CARY,BALLINTOY,5.0,THE POTTERS' CAVE,An excavation in this cave in 1934 by Jackson ...,...,NaN,26640,18/02/2009 This site appears to be undisturbed...,The scheduled area is outlined in red on the m...,BALLINTOY DEMESNE,NaN,2014-04-03 14:48:12.297,IRON AGE; PREHISTORIC; E.CHRIST.,NaN,3.0


In [12]:
df.iloc[0]

MonID                                                                        1
County                                                                     ANT
SMRNo                                                                  004:010
Council                                                                   30.0
Town                                                                       NaN
Barony                                                                    CARY
Parish                                                               BALLINTOY
Condition                                                                  2.0
SiteName                                                 STONE CIRCLE, TUMULUS
SiteSummary                  This site consists of a sub-circular mound 19....
Map12500                                                                 IG1&7
Altitude                                                                0050 F
GridRef                                             

In [13]:
len(df["MonID"].unique()), len(df)

(16695, 16695)

In [14]:
cdf = df.copy()
cdf.columns = cdf.columns.map(mapping.get)
cdf.head()

,primary_reference_number,county,None,named_location,town_or_city,building_name,building_number,spatial_accuracy_qualifier,monument_name,description,...,ownership,external_cross_reference_description,location_description,land_use_notes_value,locality,spatial_metadata_notes,date_of_authorization,cultural_period,None,comment
0,1,ANT,004:010,30.0,NaN,CARY,BALLINTOY,2.0,"STONE CIRCLE, TUMULUS",This site consists of a sub-circular mound 19....,...,NaN,NaN,18/02/2009 While there is a trail through the ...,The area to be scheduled lies within a circle ...,WHITE PARK,NaN,2009-08-19 16:39:37.033,PREHISTORIC,NaN,3.0
1,2,ANT,004:011,30.0,NaN,CARY,BALLINTOY,3.0,"CLOUGHNABOGHIL, DRUIDS STONE, CLOGHNABOGHIL, C...",On top of an enimence on a steep hill slope. T...,...,NaN,NaN,18/02/2009 The stones of this monument are und...,The area to be scheduled lies within a circle ...,LEMNAGH BEG,24/8/00\nMonitor burrow and whins and cut whi...,2009-08-19 16:39:33.080,PREHISTORIC; NEOLITHIC,NaN,3.0
2,3,ANT,004:012,30.0,NaN,CARY,BALLINTOY,1.0,CLEGNAGH CROMLECH,Situated on edge of ridge at edge of disused q...,...,NaN,NaN,28/04/2009 This was a re-visit to the site to ...,The area to be scheduled lies within a circle ...,CLEGNAGH,The owner will consider the FMW's suggestion t...,2010-01-15 11:00:56.107,NEOLITHIC; PREHISTORIC,NaN,3.0
3,4,ANT,004:013,30.0,NaN,CARY,BALLINTOY,2.0,"DRUID'S STONE, MOUNT DRUID, DRUID STONE, THE D...",On top of a knoll with superb views over White...,...,NaN,NaN,03/03/2009 The stones of this monument are und...,The area to be scheduled is a circle 15m in di...,BALLINTOY DEMESNE; MAGHERABOY,NaN,2010-01-15 11:09:17.070,PREHISTORIC; NEOLITHIC,NaN,3.0
4,5,ANT,004:014,30.0,NaN,CARY,BALLINTOY,5.0,THE POTTERS' CAVE,An excavation in this cave in 1934 by Jackson ...,...,NaN,26640,18/02/2009 This site appears to be undisturbed...,The scheduled area is outlined in red on the m...,BALLINTOY DEMESNE,NaN,2014-04-03 14:48:12.297,IRON AGE; PREHISTORIC; E.CHRIST.,NaN,3.0


In [ ]:
len((await get_monuments())['monument'])

In [ ]:
await search_monuments(text="the")

In [ ]:
import asyncio

In [15]:
coros = []
for ix, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    await create_monument_from_row(row)

NameError: name 'tqdm' is not defined